<h1 style="font-family: Times New Roman" color="blue">DigiKala Web Scraping Module</h1>

<div class="alert alert-block alert-info" style="font-family: Times New Roman; font-size:17px">
Beta version
</div>

<div class="alert alert-block alert-success" style="font-family: Times New Roman; font-size:17px">
Import libraries
</div>

In [1]:
# import libraries
from more_itertools import unique_everseen
from selenium.common.exceptions import NoSuchElementException, ElementNotInteractableException
from selenium.webdriver.common.by import By
from unidecode import unidecode
from selenium.webdriver.common.keys import Keys
from selenium import webdriver
import undetected_chromedriver as uc 
import pandas as pd
import time
import csv

<div class="alert alert-block alert-success" style="font-family: Times New Roman; font-size:17px">
Functions
</div>

In [2]:
def scrape_reviews(url, csv_name, df):
    driver.get(url)
    flag = None
    for i in range(0,20):
        try:
            button = driver.find_element(By.XPATH, "//button/div/p[@class='color-secondary-500']")
            button.click()
            print("all reviews loaded")
            flag = True
            break
        except:
            None
        
    
    # create data frame and list of reviews
    
    list_of_reviews = driver.find_elements(By.XPATH, './/article[@class="py-3 mt-0-lg d-flex ai-start br-list-vertical-no-padding-200"]')
    
    # scrape web information and format them
    for review in list_of_reviews:
        try:
            rating_persian = review.find_element(By.CSS_SELECTOR, '.p-1.radius-small.text-caption-strong.color-000.d-flex.jc-center.ai-center').text
        except:
            continue
        rating_str = unidecode(rating_persian)
        rating = float(rating_str)
        
        try:
            text = review.find_element(By.XPATH, ".//p[@class='text-body-1 color-900 mb-1 pt-3 break-words']").text
        except:
            continue
    
        if rating<4:
            sentiment=0
        else:
            sentiment=1

        new_series=pd.Series({"rating":rating, "text":text, "sentiment":sentiment})
        df = pd.concat([df, new_series.to_frame().T], ignore_index=True)
    
    df.to_csv(csv_name, mode = 'a', index = False, header = False, sep = "\t")
    return flag

In [3]:
def iterative_scraping(url, csv_name, df):
    flag = scrape_reviews(url, csv_name, df)
    print(flag)
    
    if(flag):
        list_of_footnote_pages = driver.find_elements(By.XPATH, ".//div[@class='font-body d-flex jc-center ai-center']/span[@class='d-flex w-9 h-9 jc-center ai-center radius-circle m-1 text-body2-strong pointer !hover:border border-r-700 pointer styles_PageNumberButton__yPGpm']")
        if(len(list_of_footnote_pages)>0):
            num_footnote = list_of_footnote_pages[-1].find_element(By.TAG_NAME, "span").text
            num_footnote_str = unidecode(num_footnote)
            num = int(num_footnote_str)
            
            for i in range(num):
                for i in range(0,20):
                    try:
                        button_next = driver.find_element(By.XPATH, "//div[@class='font-body  d-flex jc-center ai-center color-primary-700 pointer mr-2 pl-3 flex-row-reverse']")
                        button_next.click()
                        print("all reviews loaded")
                        flag = True
                        break
                    except:
                        None
                
                list_of_reviews = driver.find_elements(By.XPATH, './/article[@class="py-3 mt-0-lg d-flex ai-start br-list-vertical-no-padding-200"]')
    
                # scrape web information and format them
                for review in list_of_reviews:
                    try:
                        rating_persian = review.find_element(By.CSS_SELECTOR, '.p-1.radius-small.text-caption-strong.color-000.d-flex.jc-center.ai-center').text
                    except:
                        continue
                    rating_str = unidecode(rating_persian)
                    rating = float(rating_str)
        
                    try:
                        text = review.find_element(By.XPATH, ".//p[@class='text-body-1 color-900 mb-1 pt-3 break-words']").text
                    except:
                        continue
    
                    if rating<4:
                        sentiment=0
                    else:
                        sentiment=1

                    new_series=pd.Series({"rating":rating, "text":text, "sentiment":sentiment})
                    df = pd.concat([df, new_series.to_frame().T], ignore_index=True)
    
            df.to_csv(csv_name, mode = 'a', index = False, header = False, sep = "\t")
                

<div class="alert alert-block alert-success" style="font-family: Times New Roman; font-size:17px">
Keywords
</div>

In [4]:
key_words = input("Please enter keywords: ")
file_name = input("Please enter csv file name: ")

df = pd.DataFrame(columns=["rating","text","sentiment"])
headerList = ["rating","text","sentiment"]

csv_file_name = file_name + ".csv"
f = open(csv_file_name, 'w')
f.close()

Please enter keywords: موبایل شیائومی
Please enter csv file name: xiaomi_phone


<div class="alert alert-block alert-success" style="font-family: Times New Roman; font-size:17px">
Defining undetected chrome driver
</div>

In [ ]:
# define undetected chromedriver
options = uc.ChromeOptions() 
options.add_argument("--headless=new")
driver = uc.Chrome(options=options)
driver = uc.Chrome() 

driver.implicitly_wait(0.5)

url = "https://www.digikala.com/"
driver.get(url)

<div class="alert alert-block alert-success" style="font-family: Times New Roman; font-size:17px">
Search Field
</div>

In [ ]:
search_bar = driver.find_element(By.XPATH, ".//div[@class='d-flex ai-center SearchInput_SearchInput__HB9qi SearchInput_SearchInput__searchInput__CEpaj ellipsis bg-100 grow-1 radius px-2 px-4-lg text-body-2']")
search_bar.click()
input_search = driver.find_element(By.XPATH, ".//input[@class='px-2 TextField_TextField__input__hFMFl text-subtitle w-100 TextField_TextField__bwN9_ TextField_TextField--primary__IZ6Ku color-500 text-body-2 text-body-2']")

In [ ]:
input_search.send_keys(key_words)
time.sleep(0.2)
input_search.send_keys(Keys.ENTER)

In [ ]:
for i in range(10):
    driver.execute_script("window.scrollTo(100, 8250);")
    time.sleep(3)
    driver.execute_script("window.scrollTo(8250, 100);")
    time.sleep(1)

<div class="alert alert-block alert-success" style="font-family: Times New Roman; font-size:17px">
Choosing product
</div>

In [ ]:
list_of_products = driver.find_elements(By.XPATH, ".//div[@class='product-list_ProductList__item__LiiNI']")
list_products_names = []
list_products_url =[]
for product in list_of_products:
    product_name = product.find_element(By.XPATH, ".//h3[@class='ellipsis-2 text-body2-strong color-700 styles_VerticalProductCard__productTitle__6zjjN']").text
    product_url = product.find_element(By.XPATH, ".//a[@class='d-block pointer pos-relative bg-000 overflow-hidden grow-1 py-3 px-4 px-2-lg h-full-md styles_VerticalProductCard--hover__ud7aD']").get_attribute("href")
    
    if product_name in list_products_names:
        continue
    else:
        list_products_names.append(product_name)
        list_products_url.append(product_url)

In [ ]:
print("Number of products: ", len(list_products_names))

In [ ]:
for purl in list_products_url:
    iterative_scraping(purl, csv_file_name, df)

    
with open(csv_file_name, 'w') as file:
    dw = csv.DictWriter(file, delimiter=',', 
                        fieldnames=headerList)
    dw.writeheader()

file.close()